In [1]:
from random import randint, sample
from hypergraphs import *
from qf import graphs
import networkx as nx


n = 7            # Number of nodes of BH
m = 5            # Number of hyperarcs of BH
multiplicity = 6 # Maximum size of a fibre over BH

BH = hnx.Hypergraph()   # Generate BH

for i in range(m):
    k = randint(1, n)  # Number of sources
    sources = sample(range(0, n), k)
    target = randint(0, n)
    add_directed_hyperedge(BH, sources, target, f"h_{i}")

In [7]:
def gen_hyper(n, m, max_source=-1):
    """
        Returns a directed hypergraph with n nodes and m hyperarcs. Each hyperarc can have up to max_source sources (if unspecified, n).
        All hyperarcs are named "h_{i}" where i is a consecutive value.
    """
    if max_source < 0:
        max_source = n
    BH = hnx.Hypergraph()
    for i in range(m):
        k = randint(1, min(max_source, n))  # Number of sources
        sources = sample(range(0, n), k)
        target = randint(0, n)
        add_directed_hyperedge(BH, sources, target, f"h_{i}")    
    return BH

In [20]:
def left_hyper(BH, multiplicity):
    """
        Lifts the hypergraph BH, with fibers of cardinality up to multiplicity.
        The resulting graph has:
        - nodes are named (x,i) where x is a node of BH and i is in range(k), k being the size of the fiber over k
        - hyperarcs are named h_{i} where h is a hyperarc of BH and i is in range(k), k being the size of the fiber over the target of h.
        The function returns the lifted hypergraph, and a dictionary mapping the nodes of the result to the nodes of the original graph
    """
    map = {}
    B, dd = hg2g(BH) # Lift every node
    lifting = {}
    for x in B.nodes():
        if dd[x] == 0: # Node, liftable
            lifting[x]=list(range(randint(1, multiplicity)))   # Multiplicity of nodes

    H = hnx.Hypergraph() # Generate H appropriately
    for h in BH.edges():
        name = str(h)
        target = h.props['target']
        sources = set([x for x in h.elements])-set([target])
        for t in lifting[target]:
            ss = []
            for s in sources:
                s_lift = sample(lifting[s], 1)[0]
                ss += [(s, s_lift)]
            add_directed_hyperedge(H, ss, (target, t), "{}_{}".format(name, t))
    return H


In [22]:
BH = gen_hyper(3,7)
H = left_hyper(BH,3)

In [15]:
H

Hypergraph({'h_0_0': Entity(h_0_0,[(1, 0), (2, 2), (0, 0)],{'weight': 1.0, 'props': {'target': (1, 0)}}), 'h_0_1': Entity(h_0_1,[(1, 1), (2, 0), (0, 0)],{'weight': 1.0, 'props': {'target': (1, 1)}}), 'h_1_0': Entity(h_1_0,[(1, 1), (2, 0), (3, 0), (0, 0)],{'weight': 1.0, 'props': {'target': (3, 0)}}), 'h_2_0': Entity(h_2_0,[(1, 0), (2, 1)],{'weight': 1.0, 'props': {'target': (1, 0)}}), 'h_2_1': Entity(h_2_1,[(1, 1), (2, 2)],{'weight': 1.0, 'props': {'target': (1, 1)}}), 'h_3_0': Entity(h_3_0,[(1, 0), (2, 1), (3, 0), (0, 0)],{'weight': 1.0, 'props': {'target': (3, 0)}}), 'h_4_0': Entity(h_4_0,[(1, 0), (2, 0), (3, 0), (0, 0)],{'weight': 1.0, 'props': {'target': (3, 0)}}), 'h_5_0': Entity(h_5_0,[(1, 1), (2, 1), (0, 0)],{'weight': 1.0, 'props': {'target': (0, 0)}}), 'h_6_0': Entity(h_6_0,[(1, 1), (2, 0)],{'weight': 1.0, 'props': {'target': (2, 0)}}), 'h_6_1': Entity(h_6_1,[(1, 0), (2, 1)],{'weight': 1.0, 'props': {'target': (2, 1)}}), 'h_6_2': Entity(h_6_2,[(1, 0), (2, 2)],{'weight': 1.0, '

In [2]:
B, dd = hg2g(BH) # Lift every node
lifting = {}
for x in B.nodes():
    if dd[x] == 0: # Node, liftable
        lifting[x]=[i for i in range(randint(1, multiplicity))]   # Multiplicity of nodes

H = hnx.Hypergraph() # Generate H appropriately
for h in BH.edges():
    name = str(h)
    target = h.props['target']
    sources = set([x for x in h.elements])-set([target])
    for t in lifting[target]:
        ss = []
        for s in sources:
            s_lift = sample(lifting[s], 1)[0]
            ss += [(s, s_lift)]
        add_directed_hyperedge(H, ss, (target, t), "{}_{}".format(name, t))


In [3]:
rename = {}
s = {}
for h in H.edges():
    tn = h.props['target']
    t = f"x_{tn[0]}_{tn[1]}"
    if t not in rename:
        rename[t] = f"y_{len(rename)}"
    tr = rename[t]
    sources = set([f"x_{x[0]}_{x[1]}" for x in h.elements])
    zources = []
    for z in sources:
        if not z in rename:
            rename[z] = f"y_{len(rename)}"
        zources += [rename[z]]
    if tr in s:
        s[tr] += [zources]
    else:
        s[tr] = [zources]
for k, v in s.items():
    print(f"{k}' = ", " + ".join([" * ".join([vv for vv in list(x)]) for x in v]))

print()
print(rename)

y_0' =  y_0 * y_1 * y_2 * y_3 * y_4 * y_5 * y_6 + y_0 * y_3 * y_4 * y_14 + y_8 * y_0 * y_1 * y_2 * y_3
y_7' =  y_8 * y_7 * y_9 * y_10 * y_3 * y_5 * y_6 + y_7 * y_3 * y_4 * y_6 + y_8 * y_17 * y_7 * y_3 * y_18
y_11' =  y_11 * y_12 * y_2 * y_13 * y_3 * y_4 * y_5 + y_6 * y_11 * y_3 * y_15 + y_11 * y_2 * y_10 * y_3 * y_4
y_3' =  y_16 * y_11 * y_9 * y_3 * y_5
y_17' =  y_3 * y_17
y_2' =  y_3 * y_2
y_9' =  y_9 * y_3

{'x_2_0': 'y_0', 'x_4_4': 'y_1', 'x_5_1': 'y_2', 'x_0_0': 'y_3', 'x_1_0': 'y_4', 'x_6_0': 'y_5', 'x_3_3': 'y_6', 'x_2_1': 'y_7', 'x_1_2': 'y_8', 'x_5_2': 'y_9', 'x_4_3': 'y_10', 'x_2_2': 'y_11', 'x_4_1': 'y_12', 'x_3_2': 'y_13', 'x_3_1': 'y_14', 'x_1_1': 'y_15', 'x_4_0': 'y_16', 'x_5_0': 'y_17', 'x_4_2': 'y_18'}


In [111]:
s = {}
for h in BH.edges():
    tn = h.props['target']
    tr = f"x_{tn}"
    sources = set([f"x_{x}" for x in h.elements])
    if tr in s:
        s[tr] += [sources]
    else:
        s[tr] = [sources]
for k, v in s.items():
    print(f"{k}' = ", " + ".join([" * ".join([vv for vv in list(x)]) for x in v]))


x_5' =  x_2 * x_5 + x_3 * x_5 * x_0 * x_1 * x_2 * x_6
x_6' =  x_3 * x_5 * x_0 * x_1 * x_2 * x_4 * x_6
x_1' =  x_3 * x_2 * x_0 * x_1
x_7' =  x_5 * x_4 * x_1 * x_7
